# Fitting stellar parameters

The central purpose of **isochrones** is to infer the physical properties of stars given arbitrary observations. This is accomplished via the `StarModel` object.  For simplest usage, a `StarModel` is initialized with a `ModelGridInterpolator` and observed properties, provided as `(value, uncertainty)` pairs.  

## Defining a star model

First, let's generate the "observed" properties according to the model grids themselves.  Remember that `.generate()` only works with the evolution track interpolator.

In [ ]:
from isochrones.mist import MIST_EvolutionTrack, MIST_Isochrone

track = MIST_EvolutionTrack()

mass, age, feh, distance, AV = 1.0, 9.74, -0.05, 100, 0.02

true_props = track.generate(mass, age, feh, distance=distance, AV=AV)
true_props

Now, we can define a starmodel with these "observations", this time using the isochrone grid interpolator:

In [ ]:
from isochrones import StarModel, get_ichrone

mist = get_ichrone('mist')

uncs = dict(Teff=80, logg=0.1, feh=0.1, phot=0.02)
props = {p: (float(true_props[p]), uncs[p]) for p in ['Teff', 'logg', 'feh']}  # float() weirdness because of the DataFrame above
props.update({b: (float(true_props[b]), uncs['phot']) for b in 'JHK'})

# Let's also give an appropriate parallax, in mas
props.update({'parallax': (1000./distance, 0.1)})

mod = StarModel(mist, name='demo', **props)

And we can see the prior, likelihood, and posterior at the true parameters:

In [ ]:
eep = mist.get_eep(mass, age, feh, accurate=True)
pars = [eep, age, feh, distance, AV]

mod.lnprior(pars), mod.lnlike(pars), mod.lnpost(pars)

If we stray from these parameters, we can see the likelihood decrease:

In [ ]:
pars2 = [eep + 3, age - 0.05, feh + 0.02, distance, AV]
mod.lnprior(pars2), mod.lnlike(pars2), mod.lnpost(pars2)

How long does a posterior evaluation take?

In [ ]:
%timeit mod.lnpost(pars)

## Priors

As you may have noticed, we have not explictly defined any priors on our parameters.  They were defined for you, but you may wish to know what they are, and/or to change them.  

In [ ]:
mod._priors

You can sample from these priors:

In [ ]:
samples = mod.sample_from_prior(1000)
samples

Remember, these are the fit parameters:

In [ ]:
mod.param_names

Let's turn this into a dataframe, and visualize it.

In [ ]:
import pandas as pd
import holoviews as hv
import hvplot.pandas
hv.extension('bokeh')

def plot_samples(samples):
    df = pd.DataFrame(samples, columns=['eep', 'age', 'feh', 'distance', 'AV'])
    df['mass'] = mod.ic.interp_value([df.eep, df.age, df.feh], ['mass'])
    return hv.Layout([df.hvplot.hist(c).options(width=300) for c in df.columns]).cols(3)

plot_samples(samples)

Note that there are some built-in defaults here to be aware of.  The metallicity distribution is based on a local metallicity prior from SDSS, the distance prior has a maximum distance of 10kpc, and AV is flat from 0 to 1.  Now, let's change our distance prior to only go out to 1000pc, and our metallicity distribution to be flat between -2 and 0.5.

In [ ]:
from isochrones.priors import FlatPrior, DistancePrior
mod.set_prior(feh=FlatPrior((-2, 0.5)), distance=DistancePrior(1000))

In [ ]:
plot_samples(mod.sample_from_prior(1000))

Also note that the default mass prior is the Chabrier broken powerlaw, which is nifty:

In [ ]:
pd.Series(mod._priors['mass'].sample(10000), name='mass').hvplot.hist(bins=100, bin_range=(0, 5))

You can also define a metallicity prior to have a different mix of halo and (local) disk:

In [ ]:
from isochrones.priors import FehPrior

pd.Series(FehPrior(halo_fraction=0.5).sample(10000), name='feh').hvplot.hist()

## Sampling the posterior

Once you have defined your stellar model and are happy with your priors, you may either execute your optimization/sampling method of choice using the `.lnpost()` method as your posterior, or you may use the built-in **MultiNest** fitting routine with `.fit()`.  One thing to note especially is that the **MultiNest** chains get automatically created in a `chains` subdirectory from wherever you execute `.fit()`, with a basename for the files that you can access with:

In [ ]:
mod.mnest_basename

This can be changed or overwritten in two ways, which is often a good idea to avoid clashes between different fits with the same default basename.  You can either by pass an explicit `basename` keyword to `.fit()`, or you can set a name attribute, as we did when initializing this model.  OK, now we will run the fit.  This will typically take a few minutes (unless the chains for the fit have already completed, in which case it will be read in and finish quickly).

In [ ]:
mod.fit()

The posterior samples are available in the `.samples` attribute.  Note that this dataframe contains both the sampling parameters and all derived parameters from the stellar model grids at the values of those parameters. 

In [ ]:
mod.samples.head()

In [ ]:
mod.samples[mod.param_names].describe()

You can make a corner plot of any parameters you are interested in as follows:

In [ ]:
%matplotlib inline

mod.corner(mod.param_names);

There is also a convenience method to select the parameters of physical interest.

In [ ]:
mod.corner_physical();

It can also be instructive to see how the derived samples of the observed parameters compare to the observations themselves; the shortcut to this is with the `.corner_observed()` convenience method:

In [ ]:
mod.corner_observed();

This looks good, because we generated the synthetic observations directly from the same stellar model grids that we used to fit.  For real data, this is an important figure to look at to see if any of the observations appear to be inconsistent with the others, and to see if the model is a good fit to the observations.